# **Set-up**

Terminal command:

```console
pip install numpy
pip install pandas
pip install openpyxl
pip install datetime
pip install matplotlib
```

Import packages needed for notebook:

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

Ensure current working directory is in "EC1B1" folder

# **Read Excel**

In [2]:
df_spain_raw = pd.read_excel('./data/data_spain.xlsx')
df_us_raw = pd.read_excel('./data/data_united_states.xlsx')

We save dataframe in another variable to keep the raw data untouched

In [3]:
spain_1 = df_spain_raw
us_1 = df_us_raw

Inspect the dataframe

In [4]:
spain_1.head()

,International Financial Statistics (IFS),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,"Economic Activity, Industrial Production, Index","Exchange Rates, US Dollar per Domestic Currenc...","International Reserves and Liquidity, Reserves...","Prices, Consumer Price Index, All items, Index"
1,Jan 1960,NaN,0.016667,233,2.857368
2,Feb 1960,NaN,0.016667,253,2.855049
3,Mar 1960,NaN,0.016667,299,2.851573
4,Apr 1960,NaN,0.016667,326,2.853891


# **Initial Cleaning**

## For Spain:

In [5]:
spain_2 = spain_1 \
    .rename(columns={'International Financial Statistics (IFS)': 'date', 'Unnamed: 1': 'industrial_index', 'Unnamed: 2': 'nominal_exchange_rate', 'Unnamed: 3': 'reserves', 'Unnamed: 4': 'price_index'}) \
    .dropna()

In [6]:
spain_3 = spain_2
spain_3['nominal_exchange_rate'] = spain_3['nominal_exchange_rate'].apply(lambda x: 1/x)

In [8]:
spain_4 = spain_3
spain_4['date'] = spain_4['date'].apply(lambda x: datetime.strptime(x, '%b %Y'))

In [9]:
spain_5 = spain_4
spain_columns = ['industrial_index', 'nominal_exchange_rate', 'reserves', 'price_index']
for column in spain_columns:
    spain_5[column] = pd.to_numeric(spain_5[column])
    spain_5[column] = spain_5[column].round(2)

In [10]:
spain_5.head()

,date,industrial_index,nominal_exchange_rate,reserves,price_index
13,1961-01-01,18.68,60.0,536.00,2.91
14,1961-02-01,18.25,60.0,565.00,2.89
15,1961-03-01,18.42,60.0,561.55,2.87
16,1961-04-01,18.61,60.0,593.50,2.88
17,1961-05-01,19.58,60.0,643.50,2.87


In [11]:
spain_basic_cleaned = spain_5

## Using similar codes, repeat for US:

In [12]:
# Repeat first step using code from Spain
us_1 = us_1 \
    .rename(columns={'International Financial Statistics (IFS)': 'date', 'Unnamed: 1': 'reserves', 'Unnamed: 2': 'price_index'}) \
    .dropna()

# Tidy the data using similar process
us_1['date'] = us_1['date'].apply(lambda x: datetime.strptime(x, '%b %Y'))

for column in ['reserves', 'price_index']:
    us_1[column] = pd.to_numeric(us_1[column])
    us_1[column] = us_1[column].round(2)

us_basic_cleaned = us_1

In [13]:
us_basic_cleaned.head()

,date,reserves,price_index
1,1960-01-01,21478.1,13.44
2,1960-02-01,21395.7,13.48
3,1960-03-01,21344.7,13.48
4,1960-04-01,21278.0,13.53
5,1960-05-01,21234.3,13.53


# **Data manipulation**

Define real exchange rate

Create the lag variables:

In [14]:
# For Spain
spain1 = spain_basic_cleaned
spain1['nominal_exchange_rate_lag_1'] = spain1['nominal_exchange_rate'].shift(1)
spain1['price_index_lag_1'] = spain1['price_index'].shift(1)
spain1['industrial_index_lag_1'] = spain1['industrial_index'].shift(1)
spain1['industrial_index_lag_12'] = spain1['industrial_index'].shift(12)

# For US
us1 = us_basic_cleaned
us1['price_index_lag_1'] = us1['price_index'].shift(1)


In [15]:
spain1.head()

,date,industrial_index,nominal_exchange_rate,reserves,price_index,nominal_exchange_rate_lag_1,price_index_lag_1,industrial_index_lag_1,industrial_index_lag_12
13,1961-01-01,18.68,60.0,536.00,2.91,NaN,NaN,NaN,NaN
14,1961-02-01,18.25,60.0,565.00,2.89,60.0,2.91,18.68,NaN
15,1961-03-01,18.42,60.0,561.55,2.87,60.0,2.89,18.25,NaN
16,1961-04-01,18.61,60.0,593.50,2.88,60.0,2.87,18.42,NaN
17,1961-05-01,19.58,60.0,643.50,2.87,60.0,2.88,18.61,NaN
